In [3]:
# 選擇權參數

S0 = 100
K = 105
T = 1
r = 0.05
Sigma = 0.2

# Options Pricing via Monte Carlo Simulation

### Geometric Brownian Motion (G.B.M)
$$
S_t = S_0\exp\left( \left(\mu - \frac{\sigma^2}{2} \right)t + \sigma W_t\right)
$$

https://en.wikipedia.org/wiki/Geometric_Brownian_motion

In [27]:
import numpy as np

N = 100000    # simulation trials

z = np.random.standard_normal(N)
ST = S0 * np.exp((r - 0.5 * Sigma ** 2) * T + Sigma * np.sqrt(T) * z)
CT = np.maximum(ST - K, 0)
C0 = np.exp(-r * T) * np.sum(CT) / N

C0

8.0088202343802521

In [21]:
print('The present value of the European call option: %5.3f' % C0)

The present value of the European call option: 8.083


In [22]:
def Call_Value_MSC(S0, K, T, r, Sigma, N):
    '''
    Parameters:
    ===========
    S0: initial stock/index level
    K: stike price
    T: maturity date (in year factions)
    r: constant risk-free short rate (in year factions)
    Sigma: volatility factor in diffusion term (in year factions)
    N: simulation trials
    
    Returns:
    ========
    value: present value of the European call option
    '''
    from numpy import random, sqrt, exp, maximum, sum
    
    z = random.standard_normal(N)
    ST = S0 * exp((r - 0.5 * Sigma ** 2) * T + Sigma * sqrt(T) * z)
    CT = maximum(ST - K, 0)
    C0 = exp(-r * T) * sum(CT) / N
    return C0

In [25]:
Call_Value = Call_Value_MSC(S0, K, T, r, Sigma, N)
Call_Value

7.9597202836333416

# Options Pricing via BSM Model

## Call Options
$$
\begin{align}
  C(S_t, t) &= N(d_1)S_t - N(d_2) Ke^{-r(T - t)} \\
     d_1 &= \frac{1}{\sigma\sqrt{T - t}}\left[\ln\left(\frac{S_t}{K}\right) + \left(r + \frac{\sigma^2}{2}\right)(T - t)\right] \\
     d_2 &= d_1 - \sigma\sqrt{T - t} \\
\end{align}
$$

## Put Options
$$
\begin{align}
  P(S_t, t) &= Ke^{-r(T - t)} - S_t + C(S_t, t) \\
          &= N(-d_2) Ke^{-r(T - t)} - N(-d_1) S_t
\end{align}\
$$

https://en.wikipedia.org/wiki/Black%E2%80%93Scholes_model

In [8]:
def Call_Value_BSM(S0, K, T, r, Sigma):
    '''
    Parameters:
    ===========
    S0: initial stock/index level
    K: stike price
    T: maturity date (in year factions)
    r: constant risk-free short rate (in year factions)
    Sigma: volatility factor in diffusion term (in year factions)
    
    Returns:
    ========
    value: present value of the European call option
    '''
    from math import log, sqrt, exp
    from scipy import stats
    
    d1 = (log(S0 / K) + (r + 0.5 * Sigma ** 2) * T) / (Sigma * sqrt(T))
    d2 = (log(S0 / K) + (r - 0.5 * Sigma ** 2) * T) / (Sigma * sqrt(T))
    value = (S0 * stats.norm.cdf(d1, 0, 1)) - K * exp(-r * T) * stats.norm.cdf(d2, 0, 1)
    return value

In [14]:
Call_Value = Call_Value_BSM(S0, K, T, r, Sigma)
Call_Value

8.0213522351431763

In [15]:
print('The present value of the European call option: %5.3f' % Call_Value)

The present value of the European call option: 8.021


In [4]:
# function call function
def BSM_Call(S0, K, T, r, Sigma):
    '''
    Parameters:
    ===========
    S0: initial stock/index level
    K: stike price
    T: maturity date (in year factions)
    r: constant risk-free short rate (in year factions)
    Sigma: volatility factor in diffusion term (in year factions)
    
    Returns:
    ========
    value: present value of the European call option
    '''
    from math import log, sqrt, exp
    from scipy import stats
    
    d1 = BSM_d1(S0, K, T, r, Sigma)
    d2 = BSM_d2(S0, K, T, r, Sigma)
    value = (S0 * stats.norm.cdf(d1, 0, 1)) - K * exp(-r * T) * stats.norm.cdf(d2, 0, 1)
    return value

def BSM_d1(S0, K, T, r, Sigma):
    '''
    Parameters:
    ===========
    S0: initial stock/index level
    K: stike price
    T: maturity date (in year factions)
    r: constant risk-free short rate (in year factions)
    Sigma: volatility factor in diffusion term (in year factions)
    
    Returns:
    ========
    value: present value of the European call option
    '''
    from math import log, sqrt, exp
    return (log(S0 / K) + (r + 0.5 * Sigma ** 2) * T) / (Sigma * sqrt(T))

def BSM_d2(S0, K, T, r, Sigma):
    '''
    Parameters:
    ===========
    S0: initial stock/index level
    K: stike price
    T: maturity date (in year factions)
    r: constant risk-free short rate (in year factions)
    Sigma: volatility factor in diffusion term (in year factions)
    
    Returns:
    ========
    value: present value of the European call option
    '''
    from math import log, sqrt, exp
    return (log(S0 / K) + (r - 0.5 * Sigma ** 2) * T) / (Sigma * sqrt(T))

In [5]:
BSM_Call(S0, K, T, r, Sigma)

8.0213522351431763

# OOP 寫法 (Q：這樣寫好？)
用 OOP 語法改寫上面實作的內容

In [26]:
class OptionsPricing:
    # 方法：初始化屬性
    def __init__(self, S0, K, T, r, Sigma, div = 0):
        self.S0 = S0  
        self.K = K 
        self.T = T  
        self.r = r
        self.Sigma = Sigma
        self.div = div
        
    # 方法：Closed-form Solution (CF)
    def Call_Value_CF(self):
        from math import log, sqrt, exp      
        from scipy import stats
        d1 = (log(self.S0 / self.K) + (self.r + 0.5 * self.Sigma ** 2) * self.T) / (self.Sigma * sqrt(self.T))
        d2 = (log(self.S0 / self.K) + (self.r - 0.5 * self.Sigma ** 2) * self.T) / (self.Sigma * sqrt(self.T))
        C0 = (self.S0 * stats.norm.cdf(d1, 0, 1)) - K * exp(-self.r * self.T) * stats.norm.cdf(d2, 0, 1)
        #return C0
        ValMethod = 'CF'
        self.display(ValMethod, C0)
        
    # 方法：Monte Carlo Simulation (MSC)
    def Call_Value_MSC(self, N = 10000):
        from numpy import random, sqrt, exp, maximum, sum
        z = random.standard_normal(N)
        ST = self.S0 * exp((self.r - 0.5 * self.Sigma ** 2) * T + self.Sigma * sqrt(self.T) * z)
        CT = maximum(ST - self.K, 0)
        C0 = exp(-self.r * self.T) * sum(CT) / N
        #return C0
        ValMethod = 'MSC' + ' [N =' + str(N) + ']'
        self.display(ValMethod, C0)
            
    # 方法：顯示計算結果
    def display(self, ValMethod, C0):
        #print('Call (' + ValMethod + '): ' + str(C0))
        print('Call ({0}): {1}'.format(ValMethod, str(C0)))   # 新的螢幕輸出寫法 .format()

In [28]:
S0 = 100
K = 105
T = 1
r = 0.05
Sigma = 0.2
N = 100000

BS = OptionsPricing(S0, K, T, r, Sigma)
BS.Call_Value_CF()
BS.Call_Value_MSC()
BS.Call_Value_MSC(100000)

Call (CF): 8.02135223514
Call (MSC [N =10000]): 8.19034191744
Call (MSC [N =100000]): 7.98851252225


In [29]:
class OptionsPricing:
    # 方法：初始化屬性
    def __init__(self, S0, K, T, r, Sigma, Type = 'Call', Div = 0):
        self.S0 = S0  
        self.K = K 
        self.T = T  
        self.r = r
        self.Sigma = Sigma
        self.Type = Type
        self.Div = Div
        
    def calD1(self):
        from math import log, sqrt, exp
        from scipy import stats 
        res = (log(self.S0 / self.K) + (self.r + 0.5 * self.Sigma ** 2) * self.T) / (self.Sigma * sqrt(self.T))
        return res

    def calD2(self):
        from math import log, sqrt, exp
        from scipy import stats
        res = (log(self.S0 / self.K) + (self.r - 0.5 * self.Sigma ** 2) * self.T) / (self.Sigma * sqrt(self.T))
        #self.display(val)
        return res

    # 方法：Closed-form Solution (CF)
    def calCF(self):
        from math import log, sqrt, exp
        from scipy import stats
        D1 = self.calD1()
        D2 = self.calD2()
        if self.Type == 'Call':   # Call
            res = (self.S0 * stats.norm.cdf(D1, 0, 1)) - self.K * exp(-self.r * self.T) * stats.norm.cdf(D2, 0, 1)
        elif self.Type == 'Put':  # Put
            res = self.K * exp(-self.r * self.T) * stats.norm.cdf(-D2, 0, 1) - (self.S0 * stats.norm.cdf(-D1, 0, 1))
        #self.display(P0)
        return res
        
    # 方法：Monte Carlo Simulation (MSC)
    def calMSC(self, N = 10000):
        from numpy import random, sqrt, exp, maximum, sum
        z = random.standard_normal(N)
        ST = self.S0 * exp((self.r - 0.5 * self.Sigma ** 2) * self.T + self.Sigma * sqrt(self.T) * z)
        if self.Type == 'Call':   # Call
            PT = maximum(ST - self.K, 0)
        elif self.Type == 'Put':  # Put
            PT = maximum(self.K - ST, 0)
        res = exp(-self.r * self.T) * sum(PT) / N
        #self.display(P0)
        return res
            
    # 方法：顯示計算結果
    def display(self, res):
        # print(self.Type + ': ' + str(res))
        print('{Type}: {Res}'.format(Type=self.Type, Res=str(res)))    # 新型態的輸出寫法 .format()

In [30]:
Type = 'Put'
BS = OptionsPricing(S0, K, T, r, Sigma, Type)
BS.calD1()

0.10604917915283975

In [31]:
BS.calD2()

-0.09395082084716028

In [32]:
BS.calCF()

7.9004418077181455

In [33]:
res = BS.calMSC()
BS.display(res)

Put: 8.00924362667
